In [1]:
import matplotlib.pyplot as plt
import tree_utils as tu
from tqdm import tqdm
import networkx as nx
import numpy as np
import random
from sklearn.preprocessing import normalize

"""
Note:
The frontier is strictly suppposed to be used as edges only. There will be straggling nodes lying around that 
you need to be careful for. 

This is especially important for concatenating graphs. in stead of combining, it is best to place the edges into
a new graph one by one.
"""


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


'\nNote:\nThe frontier is strictly suppposed to be used as edges only. There will be straggling nodes lying around that \nyou need to be careful for. \n\nThis is especially important for concatenating graphs. in stead of combining, it is best to place the edges into\na new graph one by one.\n'

In [2]:
dims = (2, 2)
g = tu.generate_grid_graph(dims, queen=True)

g = nx.relabel_nodes(g, {node: (dims[0] * node[0]) + node[1] + 1 for node in g.nodes}, copy=False)
# tu.draw(g)

# queen_trees = sample_trees(queen, tu.uniform_random_spanning_tree, 100000)
trees = tu.enumerate_all_trees(g)

ds = [0]
tree_dict = tu.make_tree_dict(trees, ds)
counting_dict = tu.make_counting_dict(g, tree_dict, ds)

16it [00:00, 5342.21it/s]
100%|██████████████████████████████████████████████████████████████████████| 16/16 [00:00<00:00, 46538.74it/s]


In [3]:
def valid_run(snapped_edge, cc1, cc2):
    u, v = snapped_edge
    g = nx.Graph()
    g.add_edge(u, v)
    
    for edge in cc1.edges:
        u, v = edge
        g.add_edge(u, v)
        
    for edge in cc2.edges:
        u, v = edge
        g.add_edge(u, v)
        
    return nx.is_tree(g), g     

def get_ccs(g, counting_dict, snapped_edge):
    """
    """
    # u, v = tu.sample_edge_from_counts(counting_dict)
    cc1, cc2, cc1_frontier, cc2_frontier = tu.initialize(g, snapped_edge)
    target_size = len(g.nodes)/2

    while not tu.trees_correct_sizes(cc1, cc2, target_size):
        if tu.is_bottlenecked(cc1, cc1_frontier, target_size):
            tu.break_bottleneck(g, cc1, cc2, snapped_edge, cc1_frontier, cc2_frontier)

        if len(cc1.nodes) != target_size:
            tu.sample_from_frontier_to_cc(g, cc1_frontier, cc1, cc2_frontier, cc2)

        if tu.is_bottlenecked(cc2, cc2_frontier, target_size):
            tu.break_bottleneck(g, cc2, cc1, snapped_edge, cc2_frontier, cc1_frontier)

        if len(cc2.nodes) != target_size:
            tu.sample_from_frontier_to_cc(g, cc2_frontier, cc2, cc1_frontier, cc1)
    
    result, tree = valid_run(snapped_edge, cc1, cc2)
    return tree
    
edge_map = dict()
for i, edge in enumerate(g.edges):
    edge_map[edge] = i
    
tree_counter = dict()

num_runs = 10000
for i in range(num_runs):
    tree = get_ccs(g, counting_dict, (3, 4))
    edge_set = tuple(sorted([edge_map[tuple(sorted(edge))] for edge in tree.edges]))
    if edge_set in tree_counter.keys():
        tree_counter[edge_set] += 1
    else:
        tree_counter[edge_set] = 1


In [4]:
for k, v in tree_counter.items():
    tree_counter[k] = v/ num_runs

In [5]:
tree_counter

{(2, 3, 5): 0.5152, (1, 4, 5): 0.4848}

In [1]:
"""
Below are junk algorithms, mostly:
"""

'\nBelow are junk algorithms, mostly:\n'

In [3]:
"""
The properties of the boundary are that it only contains the edges that are in the xternal 
boundary of the branch, and contain no edges from the branch itself.
"""

def compute_boundary(g, branch):
    """ Computes the external boundary of the branch. This consists of boundary 
    """
    boundary = nx.Graph()
    for node in branch.nodes:
        for neighbor in g.neighbors(node):
            if neighbor not in branch.edges:
                boundary.add_edge(node, neighbor)
    return boundary
    
def add_connected_component(g, tree):
    """
    """
    leaf = pick_leaf(g, tree)
    branch = nx.Graph()
    branch.add_node(leaf)
    boundary = compute_boundary(g, branch)
    reached_tree = False
    
    while reached_tree is False:
        nxt = random.choice(list(boundary.edges))  
        
        branch.add_edge(nxt[0], nxt[1])
        
        if loop_exists(branch):
#             print("Loop exists!")
            branch.remove_edge(nxt[0], nxt[1])
            erase_singletons(branch)
        else:
            boundary = compute_boundary(g, branch)
#             print("Boundary edges after increase: ", boundary.edges)

        if nxt[0] in tree.nodes or nxt[1] in tree.nodes:
#             print("Reached Tree")
            reached_tree = True
            
    tree = tree.update(branch.edges)
    
def wilsons_modified(g):
    root = randomly_pick_root(g)
#     print("Root: ", root)
    tree = nx.Graph()
    tree.add_node(root)
    
    while tree.number_of_nodes() < g.number_of_nodes():
        add_connected_component(g, tree)
#         tu.draw(tree, delay=2)
#         print()
    return tree

# tree = wilsons_modified(g)
# tu.draw(tree)

In [4]:
def compute_boundary_node(g, branch):
    boundary = set()
    for node in branch.nodes:
        for neighbor in g.neighbors(node):
            if neighbor not in branch.nodes:
                boundary.add(node)
    return boundary

def add_connected_component_new(g, tree):
    """
    """
    leaf = pick_leaf(g, tree)
    branch = nx.Graph()
    branch.add_node(leaf)
    boundary = compute_boundary_node(g, branch)
    reached_tree = False
    
    while reached_tree is False:
        nxt_src = random.choice(list(boundary))
        nxt_dst = random.choice(list(g.neighbors(nxt_src)))
        
        branch.add_edge(nxt_src, nxt_dst)
        
        if loop_exists(branch):
#             print("Loop exists!")
            branch.remove_edge(nxt_src, nxt_dst)
            erase_singletons(branch)
        else:
            boundary = compute_boundary_node(g, branch)
#             print("Boundary edges after increase: ", boundary.edges)

        if nxt_src in tree.nodes or nxt_dst in tree.nodes:
#             print("Reached Tree")
            reached_tree = True
            
    tree = tree.update(branch.edges)
    
def wilsons_modified_node(g):
    root = randomly_pick_root(g)
#     print("Root: ", root)
    tree = nx.Graph()
    tree.add_node(root)
    
    while tree.number_of_nodes() < g.number_of_nodes():
        add_connected_component_new(g, tree)
#         tu.draw(tree, delay=2)
#         print()
    return tree

# num_trials = 100000
# data = []

# for _ in tqdm(range(num_trials)):
#     tree = wilsons_modified_node(g)

#     idx = all_trees.index(tu.tup(tree))
#     data.append(idx)
    
# tu.plot_sampled_STs(all_trees, data)

In [5]:
def erase_loopmaker_if_exists(tree, curr, nxt):
    try:
        cycle = nx.find_cycle(tree)
        tree.remove_edge(curr, nxt)
        erase_singletons(tree)
        return True
    except nx.exception.NetworkXNoCycle:
        return False
    
def cut_edges(g, tree):
    """ returns the set of cut edges of a graph
    """
    cuts = []
    for (u, v) in g.edges: 
        if u not in nx.node_connected_component(tree, v):
            cuts.append((u, v))
    return cuts
    
def add_connected_component_og(g, tree):
    """
    """
    leaf = pick_leaf(g, tree)
    curr = leaf
    
    hit_tree = False
    loopmaker = False
        
    while hit_tree is False and loopmaker is False:
        nxt = random.choice(list(g.neighbors(curr)))
            
        if tree.has_edge(curr, nxt):
            loopmaker = True
            continue
            
        tree.add_edge(curr, nxt)
        loopmaker = erase_loopmaker_if_exists(tree, curr, nxt)
        curr = nxt

        if nxt in tree.nodes():
            hit_tree = True
            

def og(g):
#     root = randomly_pick_root(g)
#     print("Root: ", root)
    tree = nx.Graph()
#     tree.add_node(root) 
    
    while tree.number_of_nodes() < g.number_of_nodes():
        add_connected_component_og(g, tree)
#         tu.draw(tree, delay=2)
#         print()
    while not nx.is_connected(tree):
#         print(len(tree.edges))
        cuts = cut_edges(g, tree)
#         print(cuts)
        edge = random.choice(cuts)
        tree.add_edge(edge[0], edge[1])
        
    return tree


# num_trials = 100000
# data = []

# for _ in tqdm(range(num_trials)):
#     tree = og(g)

#     idx = all_trees.index(tu.tup(tree))
#     data.append(idx)
    
# tu.plot_sampled_STs(all_trees, data)

In [6]:
"""
randomly pick a vertex in the loop and direct it to a new direction
"""
def nodes_from_cycle(cycle):
    nodes = set()
    for (u, v) in cycle:
        nodes.add(u)
        nodes.add(v)
    return list(nodes)

def add_branch_new(g, tree):
    leaf = pick_leaf(g, tree)
    curr = leaf
    branch = nx.Graph()
    reached_root = False
    
    while reached_root is False:
        nxt = random.choice(list(g.neighbors(curr)))
        
        branch.add_edge(curr, nxt)
        while loop_exists(branch):
            print("looping")
            print("Branch: ", branch.edges)
            cycle = nx.find_cycle(branch)
            print("Cycle: ", cycle)
            curr = random.choice(nodes_from_cycle(cycle))
            nxt = random.choice(list(g.neighbors(curr)))
            branch.add_edge(curr, nxt)
            print("curr, nxt: ", curr, nxt)
        curr = nxt
        
        if nxt in tree.nodes():
            reached_root = True
            
    tree = tree.update(branch.edges, branch.nodes)
    
def wilsons_new(g):
    root = randomly_pick_root(g)
    tree = nx.Graph()
    tree.add_node(root)
    
    while tree.number_of_nodes() < g.number_of_nodes():
        add_branch_new(g, tree)
        print("added branch")
    return tree